In [ ]:
data_lake_account_name = ''
file_system_name = ''

keyvault_name = ''

query = ''
topic = ''

In [ ]:
# News API
NEWS_API_KEY = TokenLibrary.getSecret(keyvault_name,"NEWSAPIKEY","KeyVaultLinkedService")
# Text analytics
TEXT_ANALYTICS_KEY = TokenLibrary.getSecret(keyvault_name,"TEXTANALYTICSKEY","KeyVaultLinkedService")
TEXT_ANALYTICS_ENDPOINT = TokenLibrary.getSecret(keyvault_name,"TEXTANALYTICSENDPOINT","KeyVaultLinkedService")
TEXT_ANALYTICS_REGION = TokenLibrary.getSecret(keyvault_name,"TEXTANALYTICSREGION","KeyVaultLinkedService")
# Translator
TRANSLATOR_KEY = TokenLibrary.getSecret(keyvault_name,"TRANSLATORKEY","KeyVaultLinkedService")
TRANSLATOR_ENDPOINT = TokenLibrary.getSecret(keyvault_name,"TRANSLATORENDPOINT","KeyVaultLinkedService")
TRANSLATOR_REGION = TokenLibrary.getSecret(keyvault_name,"TRANSLATORREGION","KeyVaultLinkedService")
# Twitter
TWITTER_API_KEY = TokenLibrary.getSecret(keyvault_name,"TWITTERAPIKEY","KeyVaultLinkedService")
TWITTER_API_SECRET_KEY = TokenLibrary.getSecret(keyvault_name,"TWITTERAPISECRETKEY","KeyVaultLinkedService")
TWITTER_ACCESS_TOKEN = TokenLibrary.getSecret(keyvault_name,"TWITTERACCESSTOKEN","KeyVaultLinkedService")
TWITTER_ACCESS_TOKEN_SECRET = TokenLibrary.getSecret(keyvault_name,"TWITTERACCESSTOKENSECRET","KeyVaultLinkedService")
TWITTER_API_AUTH = {
  'consumer_key': TWITTER_API_KEY,
  'consumer_secret': TWITTER_API_SECRET_KEY,
  'access_token': TWITTER_ACCESS_TOKEN,
  'access_token_secret': TWITTER_ACCESS_TOKEN_SECRET,
}
# Azure Maps
MAPS_KEY = TokenLibrary.getSecret(keyvault_name,"MAPSKEY","KeyVaultLinkedService")

In [ ]:
from datetime import datetime, date, timedelta # don't import time here. It messes with the default library
base_path = f"abfss://{file_system_name}@{data_lake_account_name}.dfs.core.windows.net/"
foldername_suffix = (datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
if topic == '':
    topic = query
subtopic = ""

language = "All"
target_languages = "English,Spanish"

from_date = ""
to_date = ""

sort_by = "popularity" # popularity, relevancy, publishedAt
qInTitle = "" 
page_size = 20

In [ ]:
import urllib.parse
import sys, time, json, requests, uuid
from azure.cosmos import CosmosClient

from dateutil.parser import parse
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential
COUNTRIES = ["ae", "ar", "at", "au", "be", "bg", "br", "ca", "ch", "cn", "co", "cu", "cz", "de", "eg", "fr", "gb", "gr", "hk", "hu", "id", "ie", "il", "in", "it", "jp", "kr", "lt", "lv", "ma", "mx", "my", "ng", "nl", "no", "nz", "ph", "pl", "pt", "ro", "rs", "ru", "sa", "se", "sg", "si", "sk", "th", "tr", "tw", "ua", "us", "ve", "za"]
LANGUAGE_CODES = {
  "All": "",
  "Afrikaans":"af",
  "Arabic":"ar",
  "Assamese":"as",
  "Bangla":"bn",
  "Bosnian(Latin)":"bs",
  "Bulgarian":"bg",
  "Cantonese(Traditional)":"yue",
  "Catalan":"ca",
  "Chinese Simplified":"zh-Hans",
  "Chinese Traditional":"zh-Hant",
  "Croatian":"hr",
  "Czech":"cs",
  "Dari":"prs",
  "Danish":"da",
  "Dutch":"nl",
  "English":"en",
  "Estonian":"et",
  "Fijian":"fj",
  "Filipino":"fil",
  "Finnish":"fi",
  "French":"fr",
  "German":"de",
  "Greek":"el",
  "Gujarati":"gu",
  "Haitian Creole":"ht",
  "Hebrew":"he",
  "Hindi":"hi",
  "Hmong Daw":"mww",
  "Hungarian":"hu",
  "Icelandic":"is",
  "Indonesian":"id",
  "Irish":"ga",
  "Italian":"it",
  "Japanese":"ja",
  "Kannada":"kn",
  "Kazakh":"kk",
  "Klingon":"tlh-Latn",
  "Klingon(plqaD)":"tlh-Piqd",
  "Korean":"ko",
  "Kurdish(Central)":"ku",
  "Kurdish(Northern)":"kmr",
  "Latvian":"lv",
  "Lithuanian":"lt",
  "Malagasy":"mg",
  "Malay":"ms",
  "Malayalam":"ml",
  "Maltese":"mt",
  "Maori":"mi",
  "Marathi":"mr",
  "Norwegian":"nb",
  "Odia":"or",
  "Pashto":"ps",
  "Persian":"fa",
  "Polish":"pl",
  "Portuguese(Brazil)":"pt-br",
  "Portuguese(Portugal)":"pt-pt",
  "Punjabi":"pa",
  "Queretaro Otomi":"otq",
  "Romanian":"ro",
  "Russian":"ru",
  "Samoan":"sm",
  "Serbian(Cyrillic)":"sr-Cyrl",
  "Serbian(Latin)":"sr-Latn",
  "Slovak":"sk",
  "Slovenian":"sl",
  "Spanish":"es",
  "Swahili":"sw",
  "Swedish":"sv",
  "Tahitian":"ty",
  "Tamil":"ta",
  "Telugu":"te",
  "Thai":"th",
  "Tongan":"to",
  "Turkish":"tr",
  "Ukrainian":"uk",
  "Urdu":"ur",
  "Vietnamese":"vi",
  "Welsh":"cy",
  "Yucatec Maya":"yua"
}

In [ ]:
from pyspark.sql.functions import *

from_date = ''

try:
    sql_str = 'select max(published_date) from articles_data where query =  ' + '\'' + str(query) + '\''
    df_maxdate = spark.sql(sql_str).first()
    from_date = df_maxdate[0]
except:
    from_date = ''

In [ ]:
config = {}
config["q"] = query
language_codes = [LANGUAGE_CODES.get(key) for key in language.split(",") if key != "All"]
if language_codes:
    if language_codes[0] in ["ar","de","en","es","fr","he","it","nl","no","pt","ru","se","ud","zh"]:
        config["language"] = language_codes[0]# ','.join(language_codes) # take the first language only as the API does not support multiple values
config["from"] = from_date
config["to"] = to_date
config["sortBy"] = sort_by
config["qInTitle"] = qInTitle
config["pageSize"] = page_size
config["apiKey"] = NEWS_API_KEY
target_languages = [LANGUAGE_CODES.get(lang, "") for lang in target_languages.split(",")]
if "en" not in target_languages:
    target_languages.append("en") # always include english in target languages

In [ ]:
from pyspark.sql import Row

def build_query(config, page=1):
    query = "http://newsapi.org/v2/everything?"
    for k,v in config.items():
        if v is None or v == "":
            continue
        if k is 'q':
            v = v.replace("#","")
            v = urllib.parse.quote_plus(v)
            query += f"{k}={v}&"
            continue        
        query += f"{k}={v}&"
    query += f"page={page}"
    return query


query = build_query(config)
print(query)

page = 1
all_articles = []
while True:
    url = build_query(config, page)
    print(f"Getting page {page}...")
    response = requests.get(url).json()
    articles = response.get("articles")
    if response["status"] == "error":
        break
    if not articles:
        break
    if page > page_size: 
        break

    all_articles.extend(articles)
    
    page += 1


In [ ]:
df_articles = spark.createDataFrame(Row(**x) for x in all_articles)
file_path = base_path + 'NewsRawData/' + config["q"]  + '_' + str(topic) + '_' + foldername_suffix
df_articles.write.format('json').save(file_path)

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

file_path = base_path + 'NewsRawData/' + config["q"]  + '_' + str(topic) + '_' + foldername_suffix
articles_df = spark.read.load(file_path, format='json')

In [ ]:
from urllib.parse import urlparse
import hashlib 

def get_month_year(publishedAt): 
    return (str(parse(publishedAt).month) + "_"+str(parse(publishedAt).year))
month_year_udf = udf(lambda publishedAt: get_month_year(publishedAt), returnType=StringType())

def get_domain(url): 
    return (urlparse(url).netloc)
domain_udf = udf(lambda url: get_domain(url), returnType=StringType())

def get_id(url,query): 
    return(hashlib.sha256((str(url) + str(query)).encode('utf-8')).hexdigest())
id_udf = udf(lambda url,query: get_id(url,query), returnType=StringType())

def get_title(title,content): 
    titleLength = len(content)
    if titleLength>80:
        titleLength=79
    if title is None:
        title=content[0:titleLength]
    
    title=title[0].upper()+title[1:]
    i=title.find(' ', titleLength)
    if i==-1:
        title=title
    else:
        title=title[0:i].replace(' ,',',').rstrip(' ').rstrip(',')
    return(title)
title_udf = udf(lambda title,content: get_title(title,content), returnType=StringType())

def get_inserted_date(publishedAt):
    dt2 = datetime.now()
    ts= int(time.mktime(dt2.timetuple()))
    at=dt2.strftime("%m/%d/%Y, %H:%M:%S %Z")
    return(at)
inserted_date_udf = udf(lambda publishedAt: get_inserted_date(publishedAt), returnType=StringType())

In [ ]:
from pyspark.sql.functions import *

df = articles_df.select(['url','author','urlToImage','description','content', 'publishedAt','title' \
                        ,col("source.Name").alias("sourceName")]) \
                .withColumn("query", lit(config["q"])) \
                .withColumn("topickey", lit(topic)) \
                .withColumn("subtopic", lit(subtopic)) \
                .withColumn("document_type", lit('news_article')) \
                .withColumn('month_year', month_year_udf(lower(col("publishedAt")))) \
                .withColumn('title', title_udf(col("title"),col("content"))) \
                .withColumn('domainname', domain_udf(col("url"))) \
                .withColumn('id', id_udf(col("url"),lit(config["q"]))) \
                .withColumn('inserted_datetime', inserted_date_udf(col("publishedAt"))) \
                .withColumn('published_date',to_date(to_timestamp(col("publishedAt"),"yyyy-MM-dd'T'HH:mm:ss'Z'"),'yyyy-MM-dd'))
               
                
df.write.mode("append").saveAsTable("articles_data")

In [ ]:
import re
import sys
import copy
import tweepy
import urllib.parse
import sys, time, json, requests, uuid
from tweepy import API
from tweepy import Cursor
from tweepy import OAuthHandler
from dateutil.parser import parse
from datetime import datetime, date, timedelta # don't import time here. It messes with the default library
from azure.cosmos import CosmosClient
from azure.core.credentials import AzureKeyCredential
from azure.ai.textanalytics import TextAnalyticsClient

def get_translation(inp_text, to_languages):
    """
    Params:
    inp_text: text to be translated
    to_languages: list of languages to translate to
    Returns: {lang_code: translation}, language code of the original text
    Call to translator cognitive service detects language and translates to the target languages. 
    Result is a dictionary of language codes to translated text, along with the language detected.
    """
    # Translator setup
    translator_path = "/translate"
    translator_url = TRANSLATOR_ENDPOINT + translator_path
    params = {
    "api-version": "3.0",
    "to": to_languages
    }
    headers = {
    'Ocp-Apim-Subscription-Key': TRANSLATOR_KEY,
    'Ocp-Apim-Subscription-Region': TRANSLATOR_REGION,
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())
    }
    # create and send request
    body = [{
    'text': inp_text
    }]
    request = requests.post(translator_url, params=params, headers=headers, json=body)
    response = request.json()
    
    try:
        from_language = response[0]["detectedLanguage"]["language"]
        translations = response[0]["translations"]
        res = {} 
        for trans in translations:
            res[trans['to']] = trans['text']
            return res, from_language 
    except Exception as err:
        print("Encountered an exception. {}".format(err))
        return err

def authenticate_client():
    """
    Returns: text analytics client
    """
    ta_credential = AzureKeyCredential(TEXT_ANALYTICS_KEY)
    text_analytics_client = TextAnalyticsClient(
            endpoint=TEXT_ANALYTICS_ENDPOINT, 
            credential=ta_credential)
    return text_analytics_client
text_analytics_client = authenticate_client()

def get_sentiment(inp_text):
    documents = [inp_text]
    response = text_analytics_client.analyze_sentiment(documents = documents)[0]  
    try:
        overallscore = response.confidence_scores.positive + (0.5*response.confidence_scores.neutral) 
        return response.sentiment, overallscore
    except Exception as err:
        print("Encountered Sentiment exception. {}".format(err))
        return "Neutral",0.5
def get_ner(inp_text):
    try:
        documents = [inp_text]
        result = text_analytics_client.recognize_entities(documents = documents)[0]  
        return [{"text": x.text, "category": x.category, "subcategory": x.subcategory, "length": x.length, "offset": x.offset, "confidence_score": x.confidence_score} for x in result.entities]
    except Exception as err:
        print("Encountered NER exception. {}".format(err))
    return []
def get_key_phrases(inp_text):
    try:
      documents = [inp_text]
      response = text_analytics_client.extract_key_phrases(documents = documents)[0] 
      if not response.is_error:
          return response.key_phrases
      else:
          print(response.id, response.error)
    except Exception as err:
      print("Encountered Translation exception. {}".format(err))
    return []

In [ ]:
import ast
from pyspark.sql import Row

schema = StructType([
    StructField("Translations", MapType(StringType(),StringType()), False),
    StructField("QueryLanguage", StringType(), False)])

def get_translations(originaltext,target_languages): 
    translations_text, query_language = get_translation(originaltext, ast.literal_eval(target_languages))
    return Row('Translations', 'QueryLanguage')(translations_text, query_language)

get_translations_udf = udf(lambda originaltext,target_languages: get_translations(originaltext,target_languages),returnType=schema)


In [ ]:
df_translations = df.select(['id','title','description','content']) \
                .withColumn('Ttitle1', get_translations_udf(col("title"),lit(str(target_languages)))) \
                .withColumn('original_language', col("Ttitle1.QueryLanguage")) \
                .select(explode(col("Ttitle1.Translations")).alias("language","TText"),'id','title','description','content','original_language') \
                .withColumn("field", lit('Title')) \
                .withColumn('created_datetime', inserted_date_udf(col("id"))) 

df_translations = df_translations.union(df_translations.select('*') \
                .withColumn('TDescr1', get_translations_udf(col("description"),lit(str(target_languages)))) \
                .withColumn('original_language', col("TDescr1.QueryLanguage")) \
                .select(explode(col("TDescr1.Translations")).alias("language","TText"),'id','title','description','content','original_language') \
                .withColumn("field", lit('Description')) \
                .withColumn('created_datetime', inserted_date_udf(col("id"))) 
                )
df_translations = df_translations.union(df_translations.select('*') \
                .withColumn('TContent1', get_translations_udf(col("content"),lit(str(target_languages)))) \
                .withColumn('original_language', col("TContent1.QueryLanguage")) \
                .select(explode(col("TContent1.Translations")).alias("language","TText"),'id','title','description','content','original_language') \
                .withColumn("field", lit('Content')) \
                .withColumn('created_datetime', inserted_date_udf(col("id"))) 
                )
df_translations = df_translations.select('id','title','description','content','original_language','created_datetime','language','field','TText')

df_translations.write.mode("append").saveAsTable("articles_translations")

In [ ]:
from pyspark.sql import Row

schema = MapType(StringType(),ArrayType(MapType(StringType(),StringType())))
        
def get_ners(original_language,language,content,TContent):
    named_entities = []
    if language != "en":
        if TContent !='':
            named_entities = get_ner(content)
            named_entity_obj = {language: named_entities}
        else:
            named_entity_obj = {language: named_entities}
    else:
        named_entities = get_ner(content) 
        named_entity_obj = {"en": named_entities}
    return(named_entity_obj)

get_ner_udf = udf(lambda original_language,language,content,TContent: get_ners(original_language,language,content,TContent),returnType=schema)

df_ner = df_translations.filter(df_translations.field == 'Content') \
                .select(['id','original_language','language','content','TText']) \
                .withColumn('NER', get_ner_udf(col("original_language"),col("language"),col("content"),col("TText"))) \
                .withColumn('created_datetime', inserted_date_udf(col("id"))) \
                .select(explode(col("NER")).alias("language","ner1"),'id','original_language','created_datetime') \
                .select(explode(col("ner1")).alias("ner2"),'id','created_datetime','language') \
                .select(['id','created_datetime','language',col("ner2.text").alias("value"),col("ner2.subcategory").alias("subcategory"),col("ner2.offset").alias("offset"),
                col("ner2.confidence_score").alias("confidence_score"), col("ner2.category").alias("category"),col("ner2.length").alias("length")])

df_ner.write.mode("append").saveAsTable("articles_entities")

In [ ]:
import ast
from pyspark.sql import Row

schema = StructType([
    StructField("sentiment", StringType(), False),
    StructField("sentiment_score", DecimalType(), False)])

def get_sentiments(original_language,title,TText): 
    if original_language != "en":
        sentiment, sentiment_score = get_sentiment(TText)
    else:
        sentiment, sentiment_score = get_sentiment(title)
    return Row('sentiment', 'sentiment_score')(sentiment, sentiment_score)

get_sentiments_udf = udf(lambda originaltext,title,TText: get_sentiments(originaltext,title,TText),returnType=schema)

df_sentiments = df_translations.filter(df_translations.field == 'Title') \
                .select(['id','original_language','title','TText']) \
                .withColumn('TS1', get_sentiments_udf(col("original_language"),col("title"),col("TText"))) \
                .withColumn('sentiment', col("TS1.sentiment")) \
                .withColumn('sentiment_score', col("TS1.sentiment_score")) \
                .withColumn('created_datetime', inserted_date_udf(col("id"))) \
                .select(['id','sentiment','sentiment_score','created_datetime'])
                
df_sentiments.write.mode("append").saveAsTable("articles_sentiments")